## Wokflow for Utah rock glacier coverage
Matt Olson - 02/2025

In [ ]:
# import libraries and functions
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt

import ee
import geemap 

# import json
import datetime
import os

In [ ]:
# autheniticate
ee.Authenticate()
ee.Initialize()

In [ ]:
# save build functions here for now
def s2_ndsi(image):
  index= image.normalizedDifference(['B3','B11']).rename('ndsi');
  return image.addBands(index)

def clip_to_polygon(image):
  return(image).clip(rg)

def scale_s2(img):
    return img.divide(10000)

def threshold1_image(image):
    # return image.gte(-1).And(image.lte(0.2)).selfMask()
    return image.gte(0.2).selfMask()

# Function to count pixels in an image
def count_pixels(image):
    total_pixels = image.select(0).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=rg,
        scale=20,  # Adjust scale as needed
        maxPixels=1e13
    ).get(image.bandNames().get(0))
    return ee.Feature(None, {'image_id': image.id(), 'pixel_count': total_pixels})


In [ ]:
# new combined functions


In [ ]:
# read in boundary
rg = geemap.shp_to_ee('shp/timp-rg/timp.shp')

## Sentinel-2

In [ ]:
# filter collection (initial)
start_date = '2015-08-01' # Define the date range
end_date = '2024-10-15'

# Define the 
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Filter the collection by date range and region
s2_filtered = s2.filterDate(start_date, end_date).filterBounds(rg) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                        .filter(ee.Filter.calendarRange(8, 10, 'month'))



In [1]:
# filter for coverage


import ee

# Initialize Earth Engine
ee.Initialize()

# Define the tolerance (in meters)
tol = 1000  # tolerance, in meters

# Define the ROI (Region of Interest) here
# For example, let's assume ROI is a point or polygon, replace with your actual ROI
ROI = ee.Geometry.Polygon([[[[-122.6, 37.0], [-122.5, 37.0], [-122.5, 37.1], [-122.6, 37.1]]]])

# Assume collection is an ImageCollection (replace with your actual collection)
collection = ee.ImageCollection('COPERNICUS/S2')  # Example, replace with your collection

# Function to compute coverage
def compute_coverage(img):
    # Get the intersection between the image geometry and the ROI with the given tolerance
    overlap = img.geometry().intersection(ROI, tol)
    
    # Calculate the area of the intersection and the total image area
    overlap_area = overlap.area().divide(img.geometry().area())
    
    # Set the 'coverage' property for the image
    img_with_coverage = img.set('coverage', overlap_area)
    
    return img_with_coverage

# Apply the function over the collection and filter based on coverage > 0.8
filtered_collection = collection.map(compute_coverage).filter(ee.Filter.gt('coverage', 0.8))

# Print the result (just for visualization)
print(filtered_collection.getInfo())


### NDSI

In [2]:
# map - scale values, clip to roi, and calculate ndsi 

# scale and clip imagery
s2_scale_clip = s2_filtered.map(scale_s2).map(clip_to_polygon)

# apply threshold
s2_thresh = s2_ndsi.map(threshold1_image)

In [3]:
# pixel count
# Map the function over the image collection
features = s2_thresh.map(count_pixels)

# Convert to a FeatureCollection and then to a pandas DataFrame
feature_collection = ee.FeatureCollection(features)
df = geemap.ee_to_df(feature_collection)

# Display the table
print(df)


In [6]:
# generate dataframe
from datetime import datetime

# Function to extract date from image name and convert to datetime
def extract_date(image_name):
    # Extract the first part of the image name (before the first 'T')
    date_str = image_name.split('T')[0]
    
    # Convert the string into a datetime object
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    
    return date_obj

df2 = df

# Apply the function to the 'image_name' column
df2['date'] = df2['image_id'].apply(extract_date)

# Calculate the area in square meters
df2['area_m2'] = df2['pixel_count'] * 400  # 400 m^2 per pixel (20m x 20m)
df2['area_km2'] = df['pixel_count'] * 400 / 1_000_000 # sq km

# Add a column for the day of the year
df['day_of_year'] = df['date'].dt.dayofyear



#### Plots

In [ ]:
# plot 1
# all years 

# Plotting the area (in km²) over time with color by year
plt.figure(figsize=(10, 6))

# Get unique years and plot each year with a different color
unique_years = df['date'].dt.year.unique()
for year in unique_years:
    year_data = df[df['date'].dt.year == year]
    plt.plot(year_data['day_of_year'], year_data['area_km2'], label=f'Year {year}', marker='o')

plt.title('Area (in km²) over Time by Year')
plt.xlabel('Day of Year')
plt.ylabel('Area (km²)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend(title="Year", loc="upper left")
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
# plot 2 
# show minimum value from each year

In [ ]:
# plot 3 
# show value closest to Oct 15?

In [5]:
# Map 1

Map = geemap.Map()
ndsi_params = {'min': -1, 'max': 1, 'bands':'ndsi','palette': ['000FFF', '00FFFF']}
vis_params = {'min': 0, 'max': 0.35, 'bands': ['B4', 'B3', 'B2']}
Map.centerObject(rg, 14)
Map.addLayer(s2_scale_clip.first(), vis_params, 'RGB')
Map.addLayer(s2_ndsi.first().updateMask(s2_ndsi.first().gte(0.2)), ndsi_params, 'NDSI > 0.4')
Map


In [7]:
# Map 2
Map.addLayer(timp_rg, {}, "Timpanogos rock glacier", False)
Map.centerObject(rg, 15)
vis_params = {
    'color': '000000', 
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

Map.addLayer(timp_rg.style(**vis_params), {}, "Styled vector")
Map


Map = geemap.Map()
ndsi_params = {'min': -1, 'max': 1, 'bands':'ndsi','palette': ['000FFF', '00FFFF']}
Map.centerObject(rg, 14)
Map.addLayer(s2_thresh.first(), ndsi_params, 'NDSI > 0.2')
Map


In [ ]:
# Map 3
# different day plot
Map = geemap.Map()

scene_number = 6

image_id = ee.Image(s2_scale_clip.toList(10).get(scene_number)).get('system:index').getInfo()
date_part = image_id.split('T')[0]
date_obj = datetime.datetime.strptime(date_part, '%Y%m%d')
formatted_date = date_obj.strftime('%m-%d-%Y')

vis_params = {'min': 0, 'max': 0.35, 'bands': ['B4', 'B3', 'B2']}
Map.centerObject(rg, 15)
Map.addLayer(ee.Image(s2_scale_clip.toList(10).get(scene_number)), vis_params, formatted_date)

Map

In [ ]:
# Map 3
# reducer - showing the coverage minimum of each year

In [ ]:
# 

In [ ]:
# show total summer snow cover days (summed)

### Landsat 8